<h1><center>openStreetMap项目记录报告</center></h1>

### 项目步骤
1. **数据梳理**
2. **数据处理**
3. **导进mongodb并探索**
4. **改进建议**

### 1. 数据梳理

In [10]:
import xml.etree.cElementTree as ET
import pprint
import codecs

def count_tags_type(file_name):
    iterparse = ET.iterparse(file_name,events = ("start",))
    tag_types = {}
    for event, elem in iterparse:
        if elem.tag not in tag_types.keys():
            tag_types[elem.tag] = 1
        else:
            tag_types[elem.tag] += 1
    return tag_types
    pass

file_name = "wuhan_china.osm"
json_file = '/home/bruce/Desktop/wuhan_china.osm.json'
tags_type = count_tags_type(file_name)
print tags_type

{'node': 319233, 'nd': 377367, 'bounds': 1, 'member': 8412, 'tag': 89575, 'osm': 1, 'way': 34552, 'relation': 332}


#### 通过这个代码我们发现了，这个不同的标签的数量，然后打开这个文件我们可以查看具体标签之间的关系。其中node是最多的，然后到way,前面的两个标签我觉得都是指道路，所以是比较多，然后就是relation这个标签，这个标签的子标签是member和tags这个应该是指道路之间的相互关系的。

### 2.数据处理

#### 这一部分是处理数据的过程，就是把所有xml的数据都转化为json格式内容。

#### 这里我发现这个邮政编码有一个不是数字的而是Wuhan Hankou，因此利用正则表达式找出来，然后统一修改为武汉市的邮政编码430000。而且把tags属性里面的name给整理在了一起。

In [1]:
def shape_element(element):
   
    node = {}
    node['type'] = element.tag # 自动添加这个type这个类型的数据
    # 这个if是处理Node节点的。
    if element.tag == 'node' or element.tag == 'relation' or element.tag == 'way':
        element_len = len(element.attrib)
        element_keys = element.attrib.keys()
        created = {}
        pos = {}
        node['type'] = element.tag
        for i in range(element_len):
            if element_keys[i] in CREATED:
                created[element_keys[i]] = element.attrib[element_keys[i]]
            elif element_keys[i] == 'lon' or element_keys[i] == 'lat':
                pos[element_keys[i]] = float(element.attrib[element_keys[i]])
            else:
                node[element_keys[i]] = element.attrib[element_keys[i]]
        if len(pos) != 0:
            pos_list = []
            pos_list.append(pos['lat'])
            pos_list.append(pos['lon'])
            node['pos'] = pos_list
        pass
        if len(created) != 0:
            node['created'] = created
        # 处理node之中的tags
        # if element.tag == 'node':
        #     # tags = {}
        #     if element.iter():
        #         for it in element.iter('tag'):
        #             tags[it.attrib['k']] = it.attrib['v']
        #             pass
        #     # if len(tags) != 0:
        #     #     node['tags'] = tags

        # 一起处理tags标签,将名字提取出来。

        if element.iter('tag'):
            name = {} # 把名字都放在一起
            tags = {}
            for it in element.iter('tag'):
                if 'name:' in it.attrib['k']:
                    key = it.attrib['k'].split(':')[1]
                    value = it.attrib['v']
                    name[key] = value
                elif 'addr:postcode' in it.attrib['k']: # 只检查addr:postcode这个属性
                    if not_number.search(it.attrib['v']):
                        tags['addr:postcode'] = '430000'
                        pass
                    postcode.append(it.attrib['v'])
                else:
                    tags[it.attrib['k']] = it.attrib['v']
                    pass
            if len(name) != 0:
                node['name'] = name
            if len(tags) != 0:
                node['tags'] = tags

        # 处理way之中的tags和nd标签
        if element.tag == 'way':
            nd_refs = []
            # tags = {}
            if element.iter():
                for it in element.iter('nd'):
                    nd_refs.append(it.attrib['ref']) # 将所有的ref的属性都添加到这个里面
                # if element.iter("tag"):
                #     for it in element.iter("tags"):
                #         tags[it.attrib['k']] = it.attrib['v']
            if len(nd_refs) != 0:
                node['nd_refs'] = nd_refs
            # if len(tags) != 0:
            #     node['tags'] = tags
        # 处理relation标签中的
        if element.tag == 'relation':
            members = []
            # tags = {}
            if element.iter('member'):
                for it in element.iter('member'):
                    members.append([it.attrib['ref']])
            # if element.iter("tags"):
            #     for it in element.iter("tags"):
            #         tags[it.attrib['k']] = it.attrib['v']
            if len(members) != 0:
                node['members'] = members
            # if len(tags) != 0:
            #     node['tags'] = tags

            pass
        return node

    else:
        return None

    pass

### 3.导进mongodb并探索

#### 这一部分的代码是负责把处理后的数据作为一个json文件来输出刚开始的时候，输出的并不是标准的json的文件内容格式，因此需要在前后分别加上[和]，再有就是内容之间需要用“，”来隔开的，因此也多加了","在中间，最后手动把最后一个“，”删除就是标准的json文件格式和内容的了。

In [ ]:
def process_map(file_in, pretty=False):
    # You do not need to change this file
    file_out = "/home/bruce/Desktop/{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo: # 注意了这句话是打开并写入一个文件里面的。
        fo.write(r'[')
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2) + ",\n")
                else:
                    fo.write(json.dumps(el) + "\n")
        fo.write(r']')
    return data

#### 把json文件的数据导进Mongodb，需要注意就是如果不是标准的json的文件格式或者内容的话是无法读取和导进mongodb的。

In [ ]:
def import_data(file_name):
    with open(file_name) as f:
        data = json.loads(f.read())
        for item in data:
            db.item.insert(item)
    pass

#### 把json文件导进了mongodb之后就可以直接在mongdb里面把数据读取出来了。 

In [7]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.map

#### 先查看数据的完整性。

In [15]:
db.item.find().count() # 查看所有的数据个数

354117

In [14]:
db.item.find({"type":"node"}).count() # 发现是和之前在统计标签的时候的数字是一样的。

319233

In [16]:
db.item.find({"type":"way"}).count() # 发现是和之前在统计标签的时候的数字是一样的。

34552

In [33]:
# 获取贡献用户的数量
len(db.item.distinct("created.user"))

535

In [25]:
# 贡献度最大的用户
agg = [doc for doc in db.item.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}},  
{"$sort":{"count": -1}}, {"$limit":1}])]
print agg[0]


{u'count': 111941, u'_id': u'GeoSUN'}


In [27]:
# 只出现一次的用户 
agg = [doc for doc in db.item.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}},  
{"$group":{"_id":"$count", "num_users":{"$sum":1}}},{"$sort":{"_id":1}}, 
{"$limit":1}])]
print agg[0]

{u'num_users': 103, u'_id': 1}


### 4.改进建议

#### 建议1：针对数据不规范的问题，可以在输入前设置规范规则，然后在输入数据之后需要对数据进行第二次的检查。  

好处：  
1.这样数据漏误更少，而且严密  
2.提高用户使用地图的准确性，增加用户体验  
预期问题：  
1.数据输入的工作量增大。

#### 建议2：允许用户自己发现并且修改数据

好处：  
1.使数据更加严密  
预期问题：  
1.数据管理工作难度增大

#### 总结：这次项目主要就是前期数据准备和处理比较多的时间，还有就是有些数据的处理还是不够完善，比如说tags的属性里面有挺多的name的应该把他们合在一起或者挑选有用的信息，还有就是导入了mongodb之后探索的还不够。